In [1]:
# Project simuluated HP
# Import packages

In [44]:
# Import libraries
import pandas as pd
import geopandas as gpd
# Operations on geometries
import shapely
import numpy as np
from numpy import nan
import math
import glob 
import xarray as xr
import xarray
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
import xgboost
import xgboost as xgb
import csv as csv
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn import linear_model
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor  #Perforing grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn import datasets, metrics, model_selection
from yellowbrick.regressor import residuals_plot


import scipy
from scipy import stats
import os

import matplotlib.pyplot as plt

from scipy.stats import skew
from collections import OrderedDict

import _pickle as cPickle

import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

from utils_functions.utils_data import *
from utils_functions.utils_ml import *
from utils_functions.utils_RF import *
from utils_functions.utils_plot import *

In [45]:
PATH =  '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/simulations_PREVAH/Classical/models/'

In [46]:
PATH_CH2018 = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/data_CH2018_tomodel/'
PATH_OUT = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/climate_rcp_simulations/RF_predsplit_CH2018/'
os.makedirs(PATH_OUT, exist_ok = True)

In [47]:
names_pw = os.listdir(PATH_CH2018)

In [48]:
nfiles = os.listdir(PATH)
# only select randomforest (best model-generally)

In [49]:
rf_mod = []
names_mod = []
for i in range(0, len(nfiles)):
    ifile = ["rf_Predefinesplit" in i for i in nfiles] 
    if ifile[i]:
        #print(nfiles[i])
        with open(PATH + nfiles[i], 'rb') as fid:
            model = cPickle.load(fid)
        rf_mod.append(model)
        names_mod.append(nfiles[i].split("_")[0])

In [50]:
# define periods of 40-yr as the historical I'm using
T1 = [2021,2061]
T2 = [2059, 2099]

In [51]:
def get_predictions (PATH_CH2018, rf_model, nam_mod, PATH_OUT):
    """Get the predictions based on the RF model from the historical period
       Args: PATH_CH2018: directory with the files
             nam_mod: name of the model/station"""

    colnames = ['date', 't2m','prec', 'prec7D', 'prec15D', 'prec30D', 't2m7D', 't2m15D',
       't2m30D','PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D']
    Features = ['t2m','prec', 'prec7D', 'prec15D', 'prec30D', 't2m7D', 't2m15D',
       't2m30D','PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D']
    
    basenames = os.listdir(PATH_CH2018 + nam_mod)
    name_ch = [os.path.splitext(x)[0] for x in basenames]

    
    x = readfiles(PATH_CH2018 + nam_mod)
    m = rf_model
    l_predictions = []
    
    for i in range(0, len(name_ch)):
        print(i)
      
        #print(x[i].head())
        #if (x[i].name_p.unique()[0] != nam_mod):
        #    raise Exception('Something is not matching!')
        df = x[i]
        
        df_new = df.rename({'discharge': 'PREVAH', 'dis7D': 'PREVAH7D', 'dis15D': 'PREVAH15D','dis30D': 'PREVAH30D'},axis='columns')
        df_proj= df_new[Features]
        # scale the data
        fpipeline = prepareData(df_proj)
        X_proj = fpipeline.fit_transform(df_proj)
        # Get the predictions
        predictions = m.model.predict(X_proj)
        # add the predictions to the data frame
        df_new['predictions'] = predictions
        # save the predictions
        PATH_stat = PATH_OUT + '/' + nam_mod + '/'
        os.makedirs(PATH_stat, exist_ok=True)
        print('save predictions for', name_ch[i])
        df_new.to_csv(PATH_stat + name_ch[i] + '.csv')
        
    l_predictions.append(df_new)
        
    return(l_predictions)

In [52]:
# Get the predictions for each 
#test= get_predictions(PATH_CH2018, rf_mod[0], names_mod[0], PATH_OUT)
for station in range(0, len(names_mod)):
    print(names_mod[station])
    get_predictions(PATH_CH2018, rf_mod[station], names_mod[station], PATH_OUT)
    

Blenio (OFIBLE)
0
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP85
1
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP26
2
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP45
3
save predictions for SMHI-RCA-MPIESM-EUR11-RCP85
4
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP85
5
save predictions for SMHI-RCA-MPIESM-EUR11-RCP45
6
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP45
7
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP26
8
save predictions for SMHI-RCA-HADGEM-EUR11-RCP45
9
save predictions for SMHI-RCA-HADGEM-EUR11-RCP85
Electra-Massa (EM)
0
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP85
1
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP26
2
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP45
3
save predictions for SMHI-RCA-MPIESM-EUR11-RCP85
4
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP85
5
save predictions for SMHI-RCA-MPIESM-EUR11-RCP45
6
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP45
7
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP26
8
save predictions for 